In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import time

In [91]:
s=Service('./app/static/ressources/chromedriver.exe')

## Images won't load in the webdriver to prevent wasting resources
option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs['profile.default_content_settings'] = {'images': 2}
chrome_prefs['profile.managed_default_content_settings'] = {'images': 2}

## Variables
main_url = 'https://www.dofus.com/fr/mmorpg/communaute/ladder'
ladder_xp = {'class_name' : 'ak-section-general',
            'filter_id':'servers_222'}
ladder_success = {'class_name' : 'ak-section-success',
            'filter_id':'servers_222'}
ladder_pvp_solo = {'class_name' : 'ak-section-kolizeum',
            'filter_id':'type_solo'}
ladder_pvp_duel = {'class_name' : 'ak-section-kolizeum',
            'filter_id':'type_duel'}

In [101]:
def open_ladder(browser, category):
    ## Access main menu URL
    browser.get(main_url)
    time.sleep(0.5)
    close_cookie = browser.find_element(By.CLASS_NAME, "ak-refuse").click()
    time.sleep(1.5)
    access_ladder = browser.find_element(By.CLASS_NAME, category['class_name']).click()
    
def filter_ladder(browser, category):
    filter_server = browser.find_element(By.ID,category['filter_id']).click()
        

In [102]:
def access_data(category):
    ## Open browser in 1920x1080
    browser = webdriver.Chrome(service=s, options=option)
    browser.set_window_size(1920,1080)
    
    open_ladder(browser, category)
    time.sleep(1.5)
    filter_ladder(browser, category)

In [103]:
access_data(ladder_xp)